# Distributed Systems
## 2022/23

Lab 9

Nuno Preguiça, Sérgio Duarte, João Resende

# Goals

In the end of this lab you should be able to:

+ Perceber o que é o OAuth.
+ Saber registar (e configurar) uma aplicação no Mastodon.
+ Saber gerar credenciais OAuth na plataforma de gestão de aplicações do Mastodon.
+ Saber tirar partido da documentação da API REST do Mastodon.
+ Saber fazer pedidos OAuth para o Mastodon em Java utilizando a biblioteca ScribeJava.

# Secure Online Services

Most online services only allow access through secure channels, with client authentication.

+ Secure channels are provided by SSL/TLS;

What about client authentication?

How can a user provide access to its resources hosted in online services to third-party clients/apps/services, 
without sharing her credentials?

# OAUTH 

OAuth allows users to provide access to resources stored in an external service (eg., Mastodon) to be accessed securely by some other application, ***without sharing the users' credentials***.

# OAuth Application registration

Applications that wish to use user’s resources in some external service **must register*** with that service.

For example, a web application designed to access the user's own Mastodon profile will have to be a registered application at Mastodon.

• This step creates the authentication pieces for that specific application: **Client key**, **Client Secret** and  an **Access token** -- required arguments for the Mastodon API.

## Register an Application with Mastodon

* You received an authentication code for a Mastodon instance running in the University internal network (vpn.fct.unl.pt)

* Connect to the internal network
* Access to [Mastodon](http://10.170.138.52:3000) and test your account
* Create an Application token:
 * Select: Preferences > development [Directlink](http://10.170.138.52:3000/settings/applications)
  * New application
  * Choose an application name
  * The application website can be: `http://localhost`
  * In the Scopes section, check `write:statuses`
  * Save changes
* Retrieve your **Client key**, **Client Secret** and generate the **Access token**


# Mastodon

[Mastodon](https://en.wikipedia.org/wiki/Mastodon_(social_network)) is a social network similar to Twitter.

Unlike Twitter, which is centralized, Mastodon is based on a notion of federated servers (domains).

The key concepts are:

+ [Status](https://docs.joinmastodon.org/entities/Status/) - represents a *post* (or message) from a user
+ [Timeline](https://docs.joinmastodon.org/methods/timelines/) - represents a list of posts (ie., feed)
 + Depending on which timeline their are posted, *statuses* can be *public* or *private*

# Mastodon API Documentation

The full Mastodon API is extensive and well documented [here](https://docs.joinmastodon.org/api/)

It will be important familiarize your self with main Mastodon concepts, bedore trying to
use the API.

The API consists of several *REST-like* endpoints, covering many functionalities.

For each endpoint that you need to access, **pay attention to**:

+ The endpoint **path**; 
+ Which **arguments** are needed and how they are passed;
+ The return code and the request result (in JSON format, most of the time).

## Mastodon - Post a Status Example

The API methods that deal with individual *status* are found [here](https://docs.joinmastodon.org/methods/statuses/)

Check the [create](https://docs.joinmastodon.org/methods/statuses/#create) endpoint.

The important details are:
+ Path: `POST /api/v1/statuses HTTP/1.1`
+ Arguments: `status` and `visibility`
+ Result: encoded in JSON

```json
{
  "id": "103254962155278888",
  "created_at": "2019-12-05T11:34:47.196Z",
  // ...
  "content": "<p>test content</p>",
  // ...
  "application": {
    "name": "test app",
    "website": null
  },
}
```

# OAuth Requests to Mastodon in JAVA            

In Java, OAuth requests are straightforward using a third party library, such as [ScribeJava](https://github.com/scribejava/scribejava)

ScribeJava requires additional Maven dependencies in the project's pom.xml
```xml
<dependency>
    <groupId>com.github.scribejava</groupId>
    <artifactId>scribejava-apis</artifactId>
    <version>8.3.3</version>
</dependency>
<dependency>
    <groupId>com.google.code.gson</groupId>
    <artifactId>gson</artifactId>
    <version>2.10.1</version>
</dependency>
```
[Gson](https://github.com/google/gson) is JSON encoder/decoder library from Google. 

Will be used to encode operation arguments and decode the results.

# Mastodon API ScribeJava: Initialization...

```java
private static final String clientKey = "<create your own>";
private static final String clientSecret = "<create your own>";
private static final String accessTokenStr = "<create your own>";

protected OAuth20Service service;
protected OAuth2AccessToken accessToken;

service = new ServiceBuilder(clientKey).apiSecret(clientSecret).build(MastodonApi.instance());
accessToken = new OAuth2AccessToken(accessTokenStr);
```

# Mastodon API Request Pattern

```java
try {
    // Assemble endpoint url (to include path arguments, if needed)
    var endpoint_url = ...
    final OAuthRequest request = new OAuthRequest(Verb.POST, endpoint_url);

	// Add Body parameters...
    request.addBodyParam(key1, value1);
    request.addBodyParam(key2, value2);

    // Add Query parameters...
    request.addQuerystringParameter(key3, value3);

	service.signRequest(accessToken, request);
	Response response = service.execute(request))
    if( response.getCode() == HTTP_OK) {
        var json = response.getBody();
        // decode JSON result
    }
    // handle error
} catch( Exception x ) {
    ...
}
```

# Mastodon API Results
## Represented as Java Records

Create a Java record to represent the data to be decoded.

```java
public record MastodonAccount(String id, String username) {
}

public record PostStatusResult(String id, String content, String created_at, MastodonAccount account) {
}

```
***NOTE:*** The record fields ***must match exactly*** the Mastodon API results JSON property names!!!

# Maston API Results
# Decoding the JSON 

Using the utility [JSON](https://github.com/preguica/sd2223/blob/main/code/aula9/src/utils/JSON.java) class, decode
the response body.

```java
if (response.getCode() == HTTP_OK) {
	var res = JSON.decode(response.getBody(), PostStatusResult.class);
    ...
}
```

For results that are lists of records, use the following:

```java
if (response.getCode() == HTTP_OK) {
	var res = JSON.decode(response.getBody(), new TypeToken<List<PostStatusResult>>() {});
    ...
}
```

# EXERCISES

+ Try the [sample code](), after you created your Mastodon application.
+ Study the Mastodon API on how to get a post and hot to delete it.
+ Investigate how the metadata that Mastodon stores with each status, can be used to fully encode a [Message](https://github.com/preguica/sd2223/blob/main/code/aula9/src/aula9/api/java/Message.java) object, including its id, creating time, text, etc.